# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f8e80662700>
├── 'a' --> tensor([[-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530]])
└── 'x' --> <FastTreeValue 0x7f8e80662ca0>
    └── 'c' --> tensor([[-1.0254, -0.4978,  0.2666, -0.7970],
                        [ 0.7406, -1.2392, -0.6167,  0.7308],
                        [-0.7619, -0.4397, -0.1032, -0.3533]])

In [4]:
t.a

tensor([[-0.3369, -0.2854,  0.6170],
        [-0.4820,  1.1594, -0.4530]])

In [5]:
%timeit t.a

62.1 ns ± 0.0207 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f8e80662700>
├── 'a' --> tensor([[-0.3978, -0.4928, -0.0220],
│                   [ 0.3820, -0.6545, -0.0639]])
└── 'x' --> <FastTreeValue 0x7f8e80662ca0>
    └── 'c' --> tensor([[-1.0254, -0.4978,  0.2666, -0.7970],
                        [ 0.7406, -1.2392, -0.6167,  0.7308],
                        [-0.7619, -0.4397, -0.1032, -0.3533]])

In [7]:
%timeit t.a = new_value

60.3 ns ± 0.0695 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530]]),
    x: Batch(
           c: tensor([[-1.0254, -0.4978,  0.2666, -0.7970],
                      [ 0.7406, -1.2392, -0.6167,  0.7308],
                      [-0.7619, -0.4397, -0.1032, -0.3533]]),
       ),
)

In [10]:
b.a

tensor([[-0.3369, -0.2854,  0.6170],
        [-0.4820,  1.1594, -0.4530]])

In [11]:
%timeit b.a

55.3 ns ± 0.107 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-1.2008, -0.3591, -1.0047],
               [ 0.8309,  1.0417, -0.5664]]),
    x: Batch(
           c: tensor([[-1.0254, -0.4978,  0.2666, -0.7970],
                      [ 0.7406, -1.2392, -0.6167,  0.7308],
                      [-0.7619, -0.4397, -0.1032, -0.3533]]),
       ),
)

In [13]:
%timeit b.a = new_value

482 ns ± 0.154 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

831 ns ± 0.109 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.2 µs ± 39.4 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 241 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

139 µs ± 355 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f8e80662670>
├── 'a' --> tensor([[[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]],
│           
│                   [[-0.3369, -0.2854,  0.6170],
│                    [-0.4820,  1.1594, -0.4530]]])
└── 'x' --> <FastTreeValue 0x7f8e806623d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

31.9 µs ± 54.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f8e806550a0>
├── 'a' --> tensor([[-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530],
│                   [-0.3369, -0.2854,  0.6170],
│                   [-0.4820,  1.1594, -0.4530]])
└── 'x' --> <FastTreeValue 0x7f8e806bbd60>
    └── 'c' --> tensor([[-1.0254, -0.4978,  0.2666, -0.7970],
                        [ 0.7406, -1.2392, -0.6167,  0.7308],
                 

In [23]:
%timeit t_cat(trees)

30.3 µs ± 51.4 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

59 µs ± 39.9 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]],
       
               [[-0.3369, -0.2854,  0.6170],
                [-0.4820,  1.1594, -0.4530]]]),
    x: Batch(
           c: tensor([[[-1.0254, -0.4978,  0.2666, -0.7970],
                       [ 0.7406, -1.2392, -0.6167,  0.7308],
                       [-0.7619, -0.4397, -0.1032, -0.3533]],
         

In [26]:
%timeit Batch.stack(batches)

78.6 µs ± 158 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530],
               [-0.3369, -0.2854,  0.6170],
               [-0.4820,  1.1594, -0.4530]]),
    x: Batch(
           c: tensor([[-1.0254, -0.4978,  0.2666, -0.7970],
                      [ 0.7406, -1.2392, -0.6167,  0.7308],
                      [-0.7619, -0.4397, -0.1032, -0.3533],
                      [-1.0254, -0.4978,  0.2666, -0.7970],
                      [ 0.7406, -

In [28]:
%timeit Batch.cat(batches)

145 µs ± 625 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

304 µs ± 641 ns per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
